#### ***Feature Engineering and Encoding***

*Feature engineering is the process of using domain knowledge to select, modify, or create features (input variables) that enhance the performance of machine learning models.* 
*It involves transforming raw data into a format that is better suited for predictive modeling. This can include:*
1. *Creating new features: Combining or transforming existing features to create new ones that capture relevant information.*
2. *Selecting features: Choosing a subset of relevant features from a larger set to improve model performance and reduce complexity.*
3. *Transforming features: Modifying features to meet the assumptions of a model, such as normalizing data or encoding categorical variables.*


In [1]:
##Import libraries

import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [2]:
## Load the cleaned dataset

df = pd.read_csv("../data/cleaned_data.csv")
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000.0,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20.0,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500.0,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,1000.0,1000.0,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644.0,2018-11-15T03:34:21Z,2,0


#### ***Aggregate Features***

In [3]:
# Creating aggregate features

aggregate_features = df.groupby('CustomerId').agg(
    Total_Transaction_Value=('Value', 'sum'),
    Average_Transaction_Value=('Value', 'mean'),
    Transaction_Count=('TransactionId', 'count'),
    Std_Deviation_Transaction_Value=('Value', 'std')
).reset_index()

# Display the aggregate features
aggregate_features

,CustomerId,Total_Transaction_Value,Average_Transaction_Value,Transaction_Count,Std_Deviation_Transaction_Value
0,CustomerId_1,10000.0,10000.000000,1,NaN
1,CustomerId_10,10000.0,10000.000000,1,NaN
2,CustomerId_1001,30400.0,6080.000000,5,4100.243895
3,CustomerId_1002,4775.0,434.090909,11,518.805446
4,CustomerId_1003,32000.0,5333.333333,6,3945.461528
...,...,...,...,...,...
3737,CustomerId_992,32000.0,5333.333333,6,4033.195590
3738,CustomerId_993,32000.0,6400.000000,5,3781.534080
3739,CustomerId_994,225077.0,2228.485149,101,3282.616882
3740,CustomerId_996,151000.0,8882.352941,17,2619.216317


#### ***Date Features***

In [4]:
# Convert TransactionStartTime to datetime
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

# Extracting features
df['Transaction_Hour'] = df['TransactionStartTime'].dt.hour
df['Transaction_Day'] = df['TransactionStartTime'].dt.day
df['Transaction_Month'] = df['TransactionStartTime'].dt.month
df['Transaction_Year'] = df['TransactionStartTime'].dt.year

# Display the updated DataFrame with new features
df[['TransactionId', 'CustomerId', 'Transaction_Hour', 'Transaction_Day', 'Transaction_Month', 'Transaction_Year']]

,TransactionId,CustomerId,Transaction_Hour,Transaction_Day,Transaction_Month,Transaction_Year
0,TransactionId_76871,CustomerId_4406,2,15,11,2018
1,TransactionId_73770,CustomerId_4406,2,15,11,2018
2,TransactionId_26203,CustomerId_4683,2,15,11,2018
3,TransactionId_380,CustomerId_988,3,15,11,2018
4,TransactionId_28195,CustomerId_988,3,15,11,2018
...,...,...,...,...,...,...
95657,TransactionId_89881,CustomerId_3078,9,13,2,2019
95658,TransactionId_91597,CustomerId_3874,9,13,2,2019
95659,TransactionId_82501,CustomerId_3874,9,13,2,2019
95660,TransactionId_136354,CustomerId_1709,10,13,2,2019


#### ***Merging Features***

In [5]:
# Display the final DataFrame with aggregate features and extracted features
final_df = df.merge(aggregate_features, on='CustomerId', how='left')

# Remove the TransactionStartTime column
final_df = final_df.drop(columns=['TransactionStartTime'])

In [6]:
final_df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,...,PricingStrategy,FraudResult,Transaction_Hour,Transaction_Day,Transaction_Month,Transaction_Year,Total_Transaction_Value,Average_Transaction_Value,Transaction_Count,Std_Deviation_Transaction_Value
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,...,2,0,2,15,11,2018,184847.0,1553.336134,119,2073.975141
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,...,2,0,2,15,11,2018,184847.0,1553.336134,119,2073.975141
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,...,2,0,2,15,11,2018,1000.0,500.000000,2,0.000000
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,1000.0,...,2,0,3,15,11,2018,117523.0,3092.710526,38,3613.003504
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,...,2,0,3,15,11,2018,117523.0,3092.710526,38,3613.003504


#### ***Removing Unwanted Columns***

In [7]:
##drop unwanted columns

# List of columns to drop
columns_to_drop = ['BatchId', 'AccountId', 'TransactionId','SubscriptionId', 'ProductId']
final_df.drop(columns=columns_to_drop, inplace=True)

#### ***Encoding***

In [8]:

# One-Hot Encoding
one_hot_columns = ['ProductCategory']
df_one_hot = pd.get_dummies(final_df, columns=one_hot_columns, drop_first=True, dtype=int)

# Display the DataFrame after One-Hot Encoding
df_one_hot.head()

,CustomerId,ProviderId,ChannelId,Amount,Value,PricingStrategy,FraudResult,Transaction_Hour,Transaction_Day,Transaction_Month,...,Transaction_Count,Std_Deviation_Transaction_Value,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill
0,CustomerId_4406,ProviderId_6,ChannelId_3,1000.0,1000.0,2,0,2,15,11,...,119,2073.975141,0,0,0,0,0,0,0,0
1,CustomerId_4406,ProviderId_4,ChannelId_2,-20.0,20.0,2,0,2,15,11,...,119,2073.975141,0,1,0,0,0,0,0,0
2,CustomerId_4683,ProviderId_6,ChannelId_3,500.0,500.0,2,0,2,15,11,...,2,0.000000,0,0,0,0,0,0,0,0
3,CustomerId_988,ProviderId_1,ChannelId_3,1000.0,1000.0,2,0,3,15,11,...,38,3613.003504,0,0,0,0,0,0,0,1
4,CustomerId_988,ProviderId_4,ChannelId_2,-644.0,644.0,2,0,3,15,11,...,38,3613.003504,0,1,0,0,0,0,0,0


In [10]:
## Process columns to extract the Id 
columns = ['CustomerId','ProviderId','ChannelId']
for col in columns:
    df_one_hot[col] = df_one_hot[col].str.split('_').str[1]

In [11]:
df_one_hot.head()

,CustomerId,ProviderId,ChannelId,Amount,Value,PricingStrategy,FraudResult,Transaction_Hour,Transaction_Day,Transaction_Month,...,Transaction_Count,Std_Deviation_Transaction_Value,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill
0,4406,6,3,1000.0,1000.0,2,0,2,15,11,...,119,2073.975141,0,0,0,0,0,0,0,0
1,4406,4,2,-20.0,20.0,2,0,2,15,11,...,119,2073.975141,0,1,0,0,0,0,0,0
2,4683,6,3,500.0,500.0,2,0,2,15,11,...,2,0.000000,0,0,0,0,0,0,0,0
3,988,1,3,1000.0,1000.0,2,0,3,15,11,...,38,3613.003504,0,0,0,0,0,0,0,1
4,988,4,2,-644.0,644.0,2,0,3,15,11,...,38,3613.003504,0,1,0,0,0,0,0,0


In [12]:
df_one_hot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   CustomerId                          95662 non-null  object 
 1   ProviderId                          95662 non-null  object 
 2   ChannelId                           95662 non-null  object 
 3   Amount                              95662 non-null  float64
 4   Value                               95662 non-null  float64
 5   PricingStrategy                     95662 non-null  int64  
 6   FraudResult                         95662 non-null  int64  
 7   Transaction_Hour                    95662 non-null  int32  
 8   Transaction_Day                     95662 non-null  int32  
 9   Transaction_Month                   95662 non-null  int32  
 10  Transaction_Year                    95662 non-null  int32  
 11  Total_Transaction_Value             95662

#### ***Handling Missing Values***

In [18]:
df_one_hot['Std_Deviation_Transaction_Value'] = df_one_hot['Std_Deviation_Transaction_Value'].fillna(0)

In [19]:
df_one_hot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   CustomerId                          95662 non-null  object 
 1   ProviderId                          95662 non-null  object 
 2   ChannelId                           95662 non-null  object 
 3   Amount                              95662 non-null  float64
 4   Value                               95662 non-null  float64
 5   PricingStrategy                     95662 non-null  int64  
 6   FraudResult                         95662 non-null  int64  
 7   Transaction_Hour                    95662 non-null  int32  
 8   Transaction_Day                     95662 non-null  int32  
 9   Transaction_Month                   95662 non-null  int32  
 10  Transaction_Year                    95662 non-null  int32  
 11  Total_Transaction_Value             95662

In [20]:
## Export the data
df_one_hot.to_csv('encoded_data.csv', index=False)